# 1. Lendo bibliotecas

In [8]:
import numpy as np
import pandas as pd
import os

# Regressão Linear
from sklearn import datasets, linear_model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression


import seaborn as sns
%matplotlib inline
import seaborn as sns
from sklearn import metrics


# Plots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

# 2. Ler o DataSet

In [9]:
filename = 'dataset.xlsx'
if not filename in os.listdir():
    print(f'Não encontrei o arquivo {filename}')

In [10]:
dados = pd.read_excel(filename)
print(f"Temos as colunas {dados.columns}")

Temos as colunas Index(['price (£)', 'number_of_reviews', 'rating', 'category'], dtype='object')


# 3. Tratamento da coluna 'Category'
A coluna category na prática é uma hierarquia de categorias que podem se misturar. Ex. A subcategoria celulares pode estar tanto dentro de comunicação quanto de tecnologia. Nesta etapa vamos separar a categoria em categoria, subcategoria e subsubcategoria.

In [21]:
# dados['parentcategory', 'subcategory', 'subsubcategory'] = dados['category'].str.split(' > ')
dados['parentcategory'] = dados['category'].map(lambda category: category.split(' > ')[0])
dados['subcategory'] = dados['category'].map(lambda category: category.split(' > ')[1])
dados['subsubcategory'] = dados['category'].map(lambda category: category.split(' > ')[2] if 2 < len(category.split(' > ')) else "None")

dados.head()

,price (£),number_of_reviews,rating,category,parentcategory,subcategory,subsubcategory
0,3.42,15,4.9,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies,Model Trains & Railway Sets,Rail Vehicles
1,16.99,2,4.5,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies,Model Trains & Railway Sets,Rail Vehicles
2,9.99,17,3.9,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies,Model Trains & Railway Sets,Rail Vehicles
3,39.99,1,5.0,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies,Model Trains & Railway Sets,Rail Vehicles
4,32.19,3,4.7,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies,Model Trains & Railway Sets,Rail Vehicles


# 4. Análise Exploratória
O primeiro passo após ter o DataSet carregado é olhar os dados e tentar descobrir possíveis correlações. Para isso vamos plotar alguns gráficos:

**Primeiramente buscamos a utilizar os dados da funcao Describe para assim plotar os graficos!**

In [ ]:

print(dados.describe(include = 'all'))

In [ ]:
# Todos os gráficos possíveis
sns.pairplot(dados,height = 5)


**Gráficos:** Esses gráficos representam nossa análise exploratória, abaixo refizemos os gráficos que consideramos os mais importantes

In [ ]:

plt.figure(figsize = (8,6))
plt.scatter(dados.rating,dados["price (£)"], c='red', alpha=0.4)
plt.title('Rating X Preço')
plt.xlabel('Rating')
plt.ylabel('Preço em euros')
plt.show()

plt.figure(figsize = (8,6))
plt.scatter(dados.number_of_reviews, dados.rating, c='red', alpha=0.4)
plt.title('Quantidade de Reviews X Rating')
plt.xlabel('Numero de Reviews')
plt.ylabel('Rating')

plt.show()

plt.figure(figsize = (8,6))
plt.scatter(dados.number_of_reviews, dados["price (£)"], c='red', alpha=0.4)
plt.title('Quantidade de Reviews X Preço')
plt.xlabel('Numero de Reviews')
plt.ylabel('Preço em Euros')

plt.figure(figsize = (8,6))
plt.hist(dados.rating, edgecolor = 'white', density = True, color = "red")
plt.title('Avaliações')
plt.xlabel('Nivel da avaliação')

plt.show()




## Observando os gráficos
Para a análise dos produtos, nós vamos usar três principais características: preço, avaliação e número de avaliações; sendo review o target e as demais variáveis features.

A partir dos gráficos acima formulamos a seguinte hipótese: **Quanto maior a avaliação, maior o preço e quanto maior número de reviews, menor o preço do produto**.
Lógica por trás da decisão:
1. Pessoas vão entrar para reclamar se o produto for ruim, mas não necessariamente vão elogiá-lo se for bom;
2. Produtos com boas avaliações tendem a ter melhor qualidade, o que reflete em seu preço

# Tentativa #1
Neste passo vamos fazer uma regressão linear multivariada utilizando MMQ, levando em consideração a o preço em função da avaliação e número de reviews.

In [23]:
def regress(Y,X):
    '''
    Y: coluna do DataFrame utilizada como variável resposta (TARGET)
    X: coluna(s) do DataFrame utilizadas como variável(is) explicativas (FEATURES)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    
    return results

In [24]:
# Target é o preço
y = dados[['price (£)']]

# Vamos usar o preço e número de reviews como features
x = dados[['number_of_reviews', 'rating']]

# Agora vamos fazer MMQ (OLS) com a função regress
resultados = regress(y, x)
resultados.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              price (£)   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     9.974
Date:                Tue, 30 Nov 2021   Prob (F-statistic):           4.72e-05
Time:                        08:38:47   Log-Likelihood:                -40588.
No. Observations:                7986   AIC:                         8.118e+04
Df Residuals:                    7983   BIC:                         8.120e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.6675      5.583      0.120      0.905     -10.277      11.612
number_of_reviews    -0.0285      0.012     -2.365      0.018      -0.052      -0.005
rating                4.1297      1.179      3.503      0.000       1.819       6.441
==============================================================================
Omnibus:                    11485.500   Durbin-Watson:                   1.793
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4758387.250
Skew:                           8.533   Prob(JB):                         0.00
Kurtosis:                     121.359   Cond. No.                         493.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Tentativa #2
Tivemos um R-squares muito baixo na última tentativa. Agora vamos tentar adicionar mais features para aumentar esse número.

In [27]:
# Target é o preço
y_2 = dados[['price (£)']]

# Vamos usar o preço e número de reviews como features
x_2 = dados[['number_of_reviews', 'rating', 'parentcategory', 'subcategory', 'subsubcategory']]

# Agora vamos fazer MMQ (OLS) com a função regress
resultados_2 = regress(y_2, x_2)
resultados_2.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

# Tentativa #2

In [ ]:
X_cp = sm.add_constant(x)

resultados.predict(X_cp)


In [ ]:
plt.figure(figsize = (8,6))
plt.scatter(dados.rating,resultados.predict(), c='blue', alpha=0.4)
plt.title('Previsão do preço X Rating')
plt.grid(True)
plt.xlabel('Rating')
plt.ylabel('Preço em Euros')

plt.figure(figsize = (8,6))
plt.grid(True)
plt.scatter(dados.number_of_reviews,resultados.predict(), c='red', alpha=0.4)
plt.xlabel('Number of Reviews')
plt.ylabel('Preço em Libras')

plt.show()

# Tentativa #3

In [ ]:
dados.columns
x = dados[['number_of_reviews', 'rating']]
y = dados[["price (£)"]]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.8)
model = LinearRegression()
model.fit(x_train, y_train)
print("O coeficiente do nosso modelo foi treinado!, segue ele:{0}".format(model.coef_))
print("Já a nossa equação de interceptação é: {0}".format(model.intercept_))
predictions = model.predict(x_test)


plt.scatter(y_test, predictions, color = "blue", alpha = 0.4)
plt.title('Predições')
plt.show()

plt.hist(y_test - predictions, color = "green")
plt.title('Restos do nosso machine learning')
plt.show()

erro_absoluto_medio = metrics.mean_absolute_error(y_test, predictions)
print('O erro absoluto médio é {0}: '.format(erro_absoluto_medio))
erro_medio_quadrado = metrics.mean_squared_error(y_test, predictions)
print('O erro  médio quadrado é {0}: '.format(erro_medio_quadrado))
raiz_erro_medio = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print('A raiz do erro médio é {0}:'.format(raiz_erro_medio))




In [ ]:
#Base regressao logistica

logistic_regression= LogisticRegression()
logistic_regression.fit(x_train,y_train)
y_pred=logistic_regression.predict(x_test)

group_names = ['True Negative','False Positive','False Negative','True Positive']
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues', )
plt.ylim(2, 0.01)
plt.show()